In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv("spam.csv",encoding="latin1")
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
df.shape

(5572, 5)

In [4]:
df=df.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'])

In [5]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
df.rename(columns={'v1':'label','v2':'text'},inplace=True)
print(df.head())

  label                                               text
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [7]:
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [8]:
df['label']=df['label'].map({'ham':0,'spam':1})
print(df.head())

   label                                               text
0      0  Go until jurong point, crazy.. Available only ...
1      0                      Ok lar... Joking wif u oni...
2      1  Free entry in 2 a wkly comp to win FA Cup fina...
3      0  U dun say so early hor... U c already then say...
4      0  Nah I don't think he goes to usf, he lives aro...


In [9]:
!pip install nltk


In [10]:
import nltk
nltk.download('stopwords')  # Download stopwords
nltk.download('wordnet')    # Download WordNet for lemmatization


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [12]:
lemmatizer=WordNetLemmatizer()
stop_words=set(stopwords.words('english'))

In [13]:
def clean_text(text):
    text=text.lower()#text to lowercase
    text = re.sub(r'https?://\S+|www\.\S+', '', text) #remove URLs
    text = text.translate(str.maketrans('', '', string.punctuation)) #remove punctuations
    text = re.sub(r'\d+', '', text) #remove numbers
    text = ' '.join(text.split()) # remove extra spaces
    text = ' '.join([word for word in text.split() if word not in stop_words])#remove stopwords
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])#lemmatize
    return text


df['text']=df['text'].apply(clean_text)
print(df.head())
    

   label                                               text
0      0  go jurong point crazy available bugis n great ...
1      0                            ok lar joking wif u oni
2      1  free entry wkly comp win fa cup final tkts st ...
3      0                u dun say early hor u c already say
4      0           nah dont think go usf life around though


In [14]:
y=df['label']
X=df['text']

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=42)

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [18]:
vectorizer=TfidfVectorizer(max_features=5000)

In [19]:
X_train_tfidf=vectorizer.fit_transform(X_train)
X_test_tfidf=vectorizer.transform(X_test)

In [20]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,classification_report

In [21]:
model=MultinomialNB()
model.fit(X_train_tfidf,y_train)

MultinomialNB()

In [22]:
y_pred=model.predict(X_test_tfidf)

In [23]:
print("Accuracy: ",accuracy_score(y_test,y_pred))

Accuracy:  0.964824120603015


In [24]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      1202
           1       1.00      0.74      0.85       191

    accuracy                           0.96      1393
   macro avg       0.98      0.87      0.92      1393
weighted avg       0.97      0.96      0.96      1393



In [25]:
import pickle
with open('spam_classifier.pkl','wb') as model_file:
    pickle.dump(model,model_file)
    
with open('vectorizer.pkl','wb') as vectorizer_file:
    pickle.dump(vectorizer,vectorizer_file)

In [26]:
with open('spam_classifier.pkl','rb') as model_file:
    model=pickle.load(model_file)
    
with open('vectorizer.pkl','rb') as vectorizer_file:
    vectorizer=pickle.load(vectorizer_file)

In [27]:
new_emails = ["Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...","Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's","URGENT! Your Mobile No. was awarded å£2000 Bonus Caller Prize on 5/9/03 This is our final try to contact U! Call from Landline 09064019788 BOX42WR29C, 150PPM","Congratulations! You've won a lottery.", "Please find the meeting agenda attached.","Hi Team,Just a reminder that we have our weekly team meeting tomorrow at 10:00 AM in Conference Room A. Please come prepared with updates on your current tasks.Thanks,[Your Manager]"]
new_emails_vectorizer=vectorizer.transform(new_emails)
predictions=model.predict(new_emails_vectorizer)
print(predictions)

[0 1 1 0 0 0]


In [28]:
print(df['label'].value_counts())


0    4825
1     747
Name: label, dtype: int64


In [29]:
from sklearn.ensemble import RandomForestClassifier
model1=RandomForestClassifier(n_estimators=100,class_weight='balanced',random_state=42)

In [30]:
model1.fit(X_train_tfidf,y_train)

RandomForestClassifier(class_weight='balanced', random_state=42)

In [31]:
y_prediction=model1.predict(X_test_tfidf)

In [32]:
print("Accuracy: ",accuracy_score(y_test,y_prediction))

Accuracy:  0.9712849964106246


In [33]:
with open('spam_classification_random_forest.pkl','wb') as model1_file:
    pickle.dump(model1,model1_file)
    

In [34]:
with open('spam_classification_random_forest.pkl','rb') as model1_file:
    model1=pickle.load(model1_file)

In [35]:
new_emails = ["Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...","Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's","URGENT! Your Mobile No. was awarded å£2000 Bonus Caller Prize on 5/9/03 This is our final try to contact U! Call from Landline 09064019788 BOX42WR29C, 150PPM","Congratulations! You've won a lottery.", "Please find the meeting agenda attached.","Hi Team,Just a reminder that we have our weekly team meeting tomorrow at 10:00 AM in Conference Room A. Please come prepared with updates on your current tasks.Thanks,[Your Manager]"]
new_emails_vectorizer=vectorizer.transform(new_emails)
predictions=model1.predict(new_emails_vectorizer)
print(predictions)

[0 1 1 0 0 0]
